In [1]:
import pandas as pd
import numpy as np
from pulp import *
from ortoolpy import addvars, addbinvars
from io import StringIO

class Shift_read():
    def __init__(self,Shift_excel="Book1.xlsx", Shift_name="Sheet1"):
        self.Shift_excel = Shift_excel
        self.Shift_name = Shift_name
        #読み込み
        self.read_df = self.read_file()
        self.df_emp = self.read_file_emp()
        self.df_need_person = self.read_need_person_file()
        self.df_Shift = self.read_file_Shift()
        
    #テーブル
    def read_file(self):
        df_1 = pd.read_excel(self.Shift_excel,sheet_name=self.Shift_name)
        #テーブルの範囲
        df = df_1.iloc[4:17,1:33]
        df = df.fillna(False)
        df = df.replace("○",True)
        df = df.replace("◎",1)
        return df
    
    #管理人かどうか(全従業員)    
    def read_file_emp(self):
        df_emp = self.read_df.iloc[0:10,0:1] 
        df_emp.reset_index(drop=True, inplace=True)
        return df_emp
    
    #必要人数
    def read_need_person_file(self):
        df_need_person = self.read_df.iloc[10:11,1:31]
        df_need_person.reset_index(drop=True, inplace=True)
        df_need_person = df_need_person.T
        df_need_person.reset_index(drop=True, inplace=True)
        return df_need_person
    
        #シフトだけにする
    def read_file_Shift(self):
        df_Shift = self.read_df.iloc[0:10,1:31]
        df_Shift.reset_index(drop=True, inplace=True)
        df_Shift = df_Shift.T
        df_Shift.reset_index(drop=True, inplace=True)
        df_Shift["必要人数"] = self.df_need_person
        return df_Shift
    
    #管理人であるかの
    def manager(self):
        manager = []
        emp = self.df_emp.values
        for i in range(len(emp)):
            if emp[i]==1:
                manager.append(i)
        return manager
    
        
        

In [2]:
class new_Shift():
    def __init__(self,df_Shift,manager):
        self.df_Shift = df_Shift
        self.L_manager = manager
        self.need()
        self.C()
        #数理モデル
        self.m = LpProblem(sense=LpMinimize)
        self.V()
        
    def need(self):
        self.n_koma = self.df_Shift.shape[0]
        self.n_emp = self.df_Shift.shape[1]-1
        self.L_emp = list(range(self.n_emp))
    
    def C(self,C_need_person = 10,C_not_hope = 100,C_least_Shift = 1,C_need_manager = 100,C_1day_2 = 10):
        #必要人数
        self.C_need_person = C_need_person
        #希望するシフトに入れない
        self.C_not_hope = C_not_hope
        #希望するシフトに半分は入れるようにする
        self.C_least_Shift = C_least_Shift
        #マネージャーが必要は必要
        self.C_need_manager = C_need_manager
    
    def V(self):
        #該当
        self.V_App = np.array(addbinvars(self.n_koma,self.n_emp)) 
        self.df_Shift["V_need_person"] = addvars(self.n_koma)
        self.V_least_Shift = addvars(self.n_emp)
        self.df_Shift["V_need_manager"] = addvars(self.n_koma)
    
    def main_1(self):
        self.mokuteki()
        self.seigyo()
        
    def main(self):
        %time self.m.solve()
    
    def mokuteki(self):
        self.m += (self.C_need_person * lpSum(self.df_Shift.V_need_person)
              + self.C_not_hope * lpSum(self.df_Shift.apply(lambda r: lpDot(1-r[self.L_emp],self.V_App[r.name]), 1))
              + self.C_least_Shift * lpSum(self.V_least_Shift)
              + self.C_need_manager * lpSum(self.df_Shift.V_need_manager))
    def seigyo(self):
        for _,r in self.df_Shift.iterrows():
            self.m += r.V_need_person >= (lpSum(self.V_App[r.name]) - r.必要人数)
            self.m += r.V_need_person >= -(lpSum(self.V_App[r.name]) - r.必要人数)
            self.m += lpSum(self.V_App[r.name,self.L_manager]) + r.V_need_manager >= 1

        #希望の半分以上
        for j,n in enumerate((self.df_Shift.iloc[:,self.L_emp].sum(0)+1)//2):
            self.m += lpSum(self.V_App[:,j]) + self.V_least_Shift[j] >= n
    
    def result_Shift(self):
        result = np.vectorize(value)(self.V_App).astype(int)
        result = result.T
        df_result = pd.DataFrame(data=result)
        df_result = df_result.replace(1,"○")
        self.cost = value(self.m.objective)
        return df_result
    

In [3]:
k = Shift_read()
df_Shift = k.df_Shift
manager = k.manager()

In [4]:
Shift = new_Shift(df_Shift,manager)

In [5]:
Shift.main_1()
Shift.main()
result = Shift.result_Shift()

CPU times: user 6.16 ms, sys: 3.94 ms, total: 10.1 ms
Wall time: 67.5 ms


In [6]:
result

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0,0,0,○,0,0,○,0,0,0,...,○,○,0,○,0,○,0,0,○,○
1,○,○,○,0,0,0,0,○,0,0,...,0,0,0,0,0,○,○,○,0,0
2,0,0,0,0,0,0,0,0,0,0,...,○,○,0,○,0,0,0,0,○,0
3,○,○,○,○,○,0,0,0,○,○,...,0,0,○,0,0,○,0,0,0,0
4,0,0,○,0,0,○,0,0,○,0,...,0,0,○,0,○,0,○,0,0,0
5,0,0,0,0,○,○,○,○,0,0,...,0,0,0,○,○,0,0,0,○,0
6,0,0,0,0,0,0,0,0,0,0,...,0,○,○,0,○,○,0,0,0,○
7,0,○,0,0,○,0,0,○,0,0,...,○,0,0,0,○,0,0,0,0,0
8,0,0,0,0,0,○,0,0,○,0,...,0,0,○,○,0,0,0,○,0,0
9,0,0,0,0,0,0,0,0,0,0,...,○,○,0,0,0,0,○,○,0,○


In [ ]:
import pandas as pd
import numpy as np
from pulp import *
from ortoolpy import addvars, addbinvars
from io import StringIO

class Shift_optimization():
    def __init__(self,):
        

In [ ]:
import pandas as pd
import numpy as np
from pulp import *
from ortoolpy import addvars, addbinvars
from io import StringIO

In [ ]:
#テーブル
df_1 = pd.read_excel("Book1.xlsx",sheet_name="Sheet1")
df = df_1.iloc[4:17,1:33]
df = df.fillna(False)
df = df.replace("○",True)
df = df.replace("◎",1)

In [ ]:
#管理人かどうか(全従業員)
df_emp = df.iloc[0:10,0:1] 
df_emp.reset_index(drop=True, inplace=True)
#df_emp.colums = ["必要人数"]

In [ ]:
#必要人数31
df_need_person = df.iloc[10:11,1:31]
df_need_person.reset_index(drop=True, inplace=True)
df_need_person = df_need_person.T
df_need_person.reset_index(drop=True, inplace=True)

In [ ]:
#シフトだけにする
df_Shift = df.iloc[0:10,1:31]
df_Shift.reset_index(drop=True, inplace=True)
df_Shift = df_Shift.T
df_Shift.reset_index(drop=True, inplace=True)

In [ ]:
df_Shift["必要人数"] = df_need_person

In [ ]:
df_Shift

In [ ]:
#管理人であるかの
def manager(employee):
    ki = []
    emp = employee.values
    for i in range(len(emp)):
        if emp[i]==1:
            ki.append(i)
    return ki

In [ ]:
n_koma, n_emp = df_Shift.shape[0], df_Shift.shape[1]-1
L_emp = list(range(n_emp))
L_manager = manager(df_emp)
#必要人数
C_need_person = 10
C_not_hope = 100
C_least_Shift = 1
C_need_manager = 100
C_1day_2 = 10

In [ ]:
df_Shift.shape[0]

In [ ]:
#数理モデル
m = LpProblem(sense=LpMinimize)

In [ ]:
#該当
V_App = np.array(addbinvars(n_koma,n_emp)) 
df_Shift["V_need_person"] = addvars(n_koma)
V_least_Shift = addvars(n_emp)
df_Shift["V_need_manager"] = addvars(n_koma)

In [ ]:
#目的変数
"""

2,
"""
m += (C_need_person * lpSum(df_Shift.V_need_person)
      + C_not_hope * lpSum(df_Shift.apply(lambda r: lpDot(1-r[L_emp],V_App[r.name]), 1))
      + C_least_Shift * lpSum(V_least_Shift)
      + C_need_manager * lpSum(df_Shift.V_need_manager))

In [ ]:
for _,r in df_Shift.iterrows():
    m += r.V_need_person >= (lpSum(V_App[r.name]) - r.必要人数)
    m += r.V_need_person >= -(lpSum(V_App[r.name]) - r.必要人数)
    m += lpSum(V_App[r.name,L_manager]) + r.V_need_manager >= 1
    
#希望の半分以上
for j,n in enumerate((df_Shift.iloc[:,L_emp].sum(0)+1)//2):
    m += lpSum(V_App[:,j]) + V_least_Shift[j] >= n

In [ ]:
%time m.solve()

In [ ]:
R結果 = np.vectorize(value)(V_App).astype(int)
#df_Shift['結果'] = [''.join(i*j for i,j in zip(r,df_Shift.columns)) for r in R結果]

In [ ]:
result = np.vectorize(value)(V_App).astype(int)
result = result.T
result
df_result = pd.DataFrame(data=result)

In [ ]:
print(value(m.objective))
df_result = df_result.replace(1,"○")

In [ ]:
R結果

In [ ]:
import pandas as pd
import openpyxl

In [ ]:
path = 'Book2.xlsx'
with pd.ExcelWriter(path) as writer:
    df_result.to_excel(writer, sheet_name='New_Shift')

In [ ]:
df_kimi = df_Shift.copy()

In [ ]:
df_kimi["必要人数"] = df_need_person

In [ ]:
df_kimi

In [ ]:
Nコマ,N従業員 = df_kimi.shape[0],df_kimi.shape[1]-1
L従業員 = list(range(N従業員))
L管理者 = [3,5,9] # 管理者は従業員3, 5, 9
C必要人数差 = 10
C希望不可 = 100
C最低コマ数 = 1
C管理者不足 = 100
C1日2コマ = 10
m = LpProblem() # 数理モデル
V割当 = np.array(addbinvars(Nコマ, N従業員))
df_kimi['V必要人数差'] = addvars(Nコマ)
V最低コマ数 = addvars(N従業員)
df_kimi['V管理者不足'] = addvars(Nコマ)

m += (C必要人数差 * lpSum(df_kimi.V必要人数差)
    + C希望不可 * lpSum(df_kimi.apply(lambda b: lpDot(1-b[L従業員],V割当[b.name]), 1))
    + C最低コマ数 * lpSum(V最低コマ数)
    + C管理者不足 * lpSum(df_kimi.V管理者不足)) # 目的関数

for _,r in df_kimi.iterrows():
    m += r.V必要人数差 >=  (lpSum(V割当[r.name]) - r.必要人数)
    m += r.V必要人数差 >= -(lpSum(V割当[r.name]) - r.必要人数)
    m += lpSum(V割当[r.name,L管理者]) + r.V管理者不足 >= 1

for j,n in enumerate((df_kimi.iloc[:,L従業員].sum(0)+1)//2):
    m += lpSum(V割当[:,j]) + V最低コマ数[j] >= n # 希望の半分以上



In [ ]:
%time m.solve()

In [ ]:
R結果 = np.vectorize(value)(V割当).astype(int)
df_kimi['結果'] = [''.join(i*j for i,j in zip(r,df_kimi.columns)) for r in R結果]
print('目的関数', value(m.objective))
print([df_kimi['結果']])

In [ ]:
R結果